<a href="https://colab.research.google.com/github/juimdpp/cca-final-project/blob/main/CCA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CCA Project: Topic 1

## 1. Load Datasets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Load breast cancer datasets
cancer = datasets.load_breast_cancer()

In [ ]:
cancer

In [ ]:
df = pd.DataFrame(cancer.data, columns = cancer.feature_names)
print("No. of samples:", len(cancer.data), "\tNo. of features:", len(cancer.feature_names))
df

## 2. Split Data into Training Sets

### Regression with All Features

In [ ]:
cancer_x = cancer.data
cancer_y = cancer.target

# split data into training sets
cancer_x_train = cancer_x[:-20]
cancer_y_train = cancer_y[:-20]

# use last 20 samples as test sets 
cancer_x_test = cancer_x[-20:]
cancer_y_test = cancer_y[-20:]
